In [ ]:
%load_ext autoreload
%autoreload 2
import hydra
import sys
sys.path.append('../meshgraphnet')
from omegaconf import DictConfig, OmegaConf
from data.datamodule import MeshDataModule
from torch_geometric.loader import DataLoader
from tasks.CfdSequenceRegressionTask import CfdSequenceRegression
from tasks.ClothSeqenceRegressionTask import ClothSequenceRegression
from visualization.viz import generate_images
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
path = ""    # path to checkpoint of cfd model
model = CfdSequenceRegression.load_from_checkpoint(path, map_location='cuda')

cfg = {
    'dataset_name': 'cylinder_flow',
    'field': 'velocity',
    'history': False,
    'batch_size_train': 1,
    'batch_size_valid': 10,
    'data_dir': '/nfs/students/yan/data/cylinder_flow',
    'steps': [1, 10, 50, 100, 200],
    'max_step': 598              # rollout 598 steps
}
cfg = OmegaConf.create(cfg)

datamodule = MeshDataModule(cfg)
datamodule.prepare_data()
datamodule.setup()

valid_ds = datamodule.valid_ds
valid_loader = datamodule.val_dataloader()

trajectories = model.predict_trajectory(valid_loader, step=cfg.max_step)

In [ ]:
scalars = {'mse_%03d_steps' % step: 0.0 for step in cfg.steps}

num_data = len(trajectories)
for i in range(num_data):
    error = np.mean((valid_ds[i].target_velocity[:, :cfg.max_step].numpy() - trajectories[i].numpy())**2, axis=-1)
    scalars = {'mse_%03d_steps' % step:
               scalars['mse_%03d_steps' % step] + np.mean(error[:, :step])
               for step in cfg.steps}
scalars = {key: value / num_data for key, value in scalars.items()}
print(scalars)

In [ ]:
path = ""     # path to checkpoint of cloth model
model = ClothSequenceRegression.load_from_checkpoint(path, map_location='cuda')

cfg = {
    'dataset_name': 'flag_simple',
    'field': 'world_pos',
    'history': True,
    'batch_size_train': 1,
    'batch_size_valid': 20,
    'data_dir': '/nfs/students/yan/data/flag_simple',
    'steps': [1, 10, 50],
    'max_step': 50             # rollout 50 steps
}
cfg = OmegaConf.create(cfg)

datamodule = MeshDataModule(cfg)
datamodule.prepare_data()
datamodule.setup()

valid_ds = datamodule.valid_ds
valid_loader = datamodule.val_dataloader()

trajectories = model.predict_trajectory(valid_loader, step=cfg.max_step)

In [ ]:
scalars = {'mse_%03d_steps' % step: 0.0 for step in cfg.steps}

num_data = len(trajectories)
for i in range(num_data):
    error = np.mean((valid_ds[i].target_world_pos[:, :cfg.max_step].numpy() - trajectories[i].numpy())**2, axis=-1)
    scalars = {'mse_%03d_steps' % step:
               scalars['mse_%03d_steps' % step] + np.mean(error[:, :step])
               for step in cfg.steps}
scalars = {key: value / num_data for key, value in scalars.items()}
print(scalars)